In [8]:
import requests
import ssl
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
import json

In [9]:
class SSLAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        ctx.options |= 0x4  # ssl.OP_LEGACY_SERVER_CONNECT
        kwargs['ssl_context'] = ctx
        return super(SSLAdapter, self).init_poolmanager(*args, **kwargs)

In [10]:
headers = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-GB,en;q=0.9,en-US;q=0.8",
    "Cache-Control": "no-cache",
    "Connection": "keep-alive",
    "Host": "diemthi.vnanet.vn",
    "Pragma": "no-cache",
    "Referer": "https://diemthi.vnanet.vn/diem-thi/2021",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36 Edg/127.0.0.0",
    "X-Requested-With": "XMLHttpRequest",
    "sec-ch-ua": '"Not)A;Brand";v="99", "Microsoft Edge";v="127", "Chromium";v="127"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Linux"'
}

session = requests.Session()
session.mount("https://", SSLAdapter())

In [11]:
url = "https://diemthi.vnanet.vn/diem-thi/2024"

response = session.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')

with open('diemthi.html', 'w', encoding='utf-8') as f:
    f.write(soup.prettify())

location_codes = soup.find('select', id='listCity')
subjects = soup.find('select', id='listSubject')

location_code_options = location_codes.find_all('option') if location_codes else []
subject_options = subjects.find_all('option') if subjects else []

location_code_dict = {option['value']: option.text.strip() for option in location_code_options}
subject_dict = {option['value']: option.text.strip() for option in subject_options}

location_code_dict.pop("00", None)
location_code_dict.pop("65", None)
location_code_dict = {key: value.split('Sở GDĐT ')[-1] for key, value in location_code_dict.items()}
location_code_dict["60"] = location_code_dict["60"].split('Sở GD KHCN ')[-1]

location_code_json = json.dumps(location_code_dict, ensure_ascii=False, indent=4)
subject_json = json.dumps(subject_dict, ensure_ascii=False, indent=4)

In [12]:
print(location_code_json)

{
    "01": "Hà Nội",
    "02": "TP. Hồ Chí Minh",
    "03": "Hải Phòng",
    "04": "Đà Nẵng",
    "05": "Hà Giang",
    "06": "Cao Bằng",
    "07": "Lai Châu",
    "08": "Lào Cai",
    "09": "Tuyên Quang",
    "10": "Lạng Sơn",
    "11": "Bắc Kạn",
    "12": "Thái Nguyên",
    "13": "Yên Bái",
    "14": "Sơn La",
    "15": "Phú Thọ",
    "16": "Vĩnh Phúc",
    "17": "Quảng Ninh",
    "18": "Bắc Giang",
    "19": "Bắc Ninh",
    "21": "Hải Dương",
    "22": "Hưng Yên",
    "23": "Hoà Bình",
    "24": "Hà Nam",
    "25": "Nam Định",
    "26": "Thái Bình",
    "27": "Ninh Bình",
    "28": "Thanh Hoá",
    "29": "Nghệ An",
    "30": "Hà Tĩnh",
    "31": "Quảng Bình",
    "32": "Quảng Trị",
    "33": "Thừa Thiên -Huế",
    "34": "Quảng Nam",
    "35": "Quảng Ngãi",
    "36": "Kon Tum",
    "37": "Bình Định",
    "38": "Gia Lai",
    "39": "Phú Yên",
    "40": "Đắk Lắk",
    "41": "Khánh Hoà",
    "42": "Lâm Đồng",
    "43": "Bình Phước",
    "44": "Bình Dương",
    "45": "Ninh Thuận",
    

In [13]:
print(subject_json)

{
    "Toan": "Toán",
    "VatLi": "Vật lí",
    "HoaHoc": "Hóa",
    "SinhHoc": "Sinh học",
    "KHTN": "KHTN",
    "NguVan": "Ngữ văn",
    "LichSu": "Lịch sử",
    "DiaLi": "Địa lí",
    "GDCD": "GDCD",
    "KHXH": "KHXH",
    "NgoaiNgu": "Ngoại Ngữ"
}


In [14]:
with open('misc/location_code.json', 'w', encoding='utf-8') as f:
    f.write(location_code_json)

with open('misc/subject.json', 'w', encoding='utf-8') as f:
    f.write(subject_json)